# NFL Data Weekly Extraction & Insert function — 2025 Season
This notebook extract and insert weekly NFL team, player, and schedule data to SQL.

In [ ]:
#-----------------------------------#
#WEEKLY UPDATE Code
#This is what I would run every week
#-----------------------------------#


#Load the latest datat into Pandas DataFrames (selecting our Target Columns)
df_team_stats = nfl.load_team_stats().to_pandas()[target_columns_team_stats] #team game or season statistics (current season)
df_player_stats = nfl.load_player_stats().to_pandas()[target_columns_player_stats] #player game or season statistics (current season)
df_season_schedule = nfl.load_schedules(2025).to_pandas()[target_columns_schedule] #game schedules and results (current season), #This data will need updating later on with, updated scores

#Function that extracts and uploads rows of weekly nfl data
#key_cols will differ depending on which table we are inserting data to.
#key_cols:
    #nfl_team_stats: "team", "week"
    #nfl_player_stats: "player_id", "week"
def weekly_data_upload(df, week=None, column = "week", table_name = "nfl_team_stats", key_cols = ["team", "week"]):
    try:
        #1) Determine the week
        if week is None:
            week = df[column].max() #getting the max value on the column "week", i.e the latest week
            print(f"No week specified, using the latest week: {week}")
            
        df_weekly = df[df[column]==week].copy() #Making a copy DataFrame 
        print(f" Processing {len(df_weekly)} rows for week {week}")

        #2) Checking what's already in the SQL table to determine what rows we have
        try:
            df_old = pd.read_sql(f"SELECT * FROM {table_name}", engine)
            print(f"There are {len(df_old)} existing rows in {table_name}")
        except Exception:
            df_old = pd.DataFrame(columns = df_weekly.columns)
            print(f"{table_name} was not found")

        #3) Identifying new rows
        df_new_rows = df_weekly.loc[ #Checking the values in the DataFrame and making them into composite keys (.apply tuple()) and checking if they exist in the SQL talb
            ~df_weekly[key_cols].apply(lambda x: tuple(x), axis = 1).isin( #We're only displaying the values that are true, aka the values that does not exist
                df_old[key_cols].apply(lambda x: tuple(x), axis = 1)
            )
        ][df_weekly.columns]

        #4) Insert new rows to the table
        if not df_new_rows.empty:
            df_new_rows.to_sql(table_name, engine, if_exists='append', chunksize=5000, index=False) #Exporting data to SQL
            print(f"Inserted {len(df_new_rows)} new rows to {table_name}")
        else:
            print("No new rows added")
    except Exception as e:
        print(f"❌ Upload failed: {e}")

#Calling the function
weekly_data_upload(df_team_stats)